# ИНДИВИДУАЛЬНОЕ ЗАДАНИЕ НА ПРАКТИКУ

1. Собрать датасет из текстов для вашего классификатора. Обучающая выборка для бинарного
классификатора – не менее 100 записей, для мультиклассификатора – не менее 50 текстов
для каждого класса. Можно больше, но рассчитывайте, что расхождения в количестве
записей, принадлежащих каждому конкретному классу, не должно расходиться более чем
на 10% между классами считая от минимального (т.е. в одном классе может быть 50, а в
другом 60 – во втором 120% от первого по количеству записей). Тестовая выборка
формируется в соотношении с тренировочной 3:7 (чтобы не перепутать: тренировочная
больше).

2. Подготовить датасет по принципу, представленному в ЛР1 (фильтрация  токенизация 
нормализация). Одна из ключевых задач – фильтрация. В тексте должно остаться как можно
меньше часто встречающихся лексем, не представляющих значения для будущего
классификатора (широко распространенные глаголы, вводные конструкции и т.д.) Чем
тщательнее будет подготовлен датасет, тем лучше будет результат.

3. Реализовать алгоритм TF-IDF самостоятельно (НЕ использовать готовую реализацию из
scikit-learn). TD-IDF это метрика, рассчитываемая для каждой лексемы в каждом документе.
В разных документах значение TF-IDF для одной и той же лексемы будет отличаться.
Формула TF-IDF для некоторого корпуса D.

4. Собрать разреженную матрицу лексемы-документы (где документы – строки, лексемы –
столбцы). Количество столбцов соответствует всему перечню уникальных лексем (после
нормализации) из текста. Ячейки заполняются следующим образом: если лексема
присутствует в документе – ей присваивается значение TD-IDF для лексемы в данном
документе, если лексема в данном документе отсутствует – присваивается значение 0.

5. Провести разметку классов на собранных данных (присвоить каждому документу класс
согласно тому, какой именно у вас классификатор – бинарный или мульти-).

6. Применить не менее 4-х классификаторов (на Ваш выбор) из библиотеки scikit-learn.

7. Подготовить отчет.


# Введение

Цель работы: создать бинарный  классификатор текстов с использованием библиотеки scikit-learn и матрицы типа «лексемы-документы», полученной с помощью метрики TF-IDF, произвести оценку производительности классификаторов и оценку эффективности классификации.

# Ход работы

In [104]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize
from collections import Counter
import math
import sklearn
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

#Сбор данных парсер собрирает тексты песен из веб-сайта lyrics.com. 
#Эти песни разделеям на два жанра: "pop" и "rap".
BASE_URL = "https://www.lyrics.com"

# Задаем ссылки для сбора песен в жанрах "pop" и "rap"
pop_links = [
    '/lyric/31566639/Eminem/Guts+Over+Fear',
    '/lyric/35378068/Eminem/Kamikaze',
    '/lyric/31566636/Eminem/Fine+Line',
    '/lyric/35368562/Eminem/Not+Alike',
    '/lyric/35368565/Eminem/Normal',
    '/lyric/35368563/Eminem/Stepping+Stone',
    '/lyric/35368568/Eminem/Greatest',
    '/lyric/35368567/Eminem/Lucky+You',
    '/lyric/35368569/Eminem/The+Ringer',
    '/lyric/34675916/Eminem/Believe',
    '/lyric/34675914/Eminem/Untouchable',
    '/lyric/34675913/Eminem/River'
]

rap_links = [
    '/lyric/34675911/Eminem/Remind+Me',
    '/lyric/34675908/Eminem/Bad+Husband',
    '/lyric/34675907/Eminem/Tragic+Endings',
    '/lyric/35368563/Eminem/Stepping+Stone'
    '/lyric/34675906/Eminem/Framed',
    '/lyric/34675905/Eminem/Nowhere+Fast',
    '/lyric/34675904/Eminem/Heat',
    '/lyric/34675903/Eminem/Offended',
    '/lyric/34675902/Eminem/Need+Me',
    '/lyric/34675901/Eminem/In+Your+Head',
    '/lyric/34675900/Eminem/Castle',
    '/lyric/31566644/Eminem/Shady+XV',
    '/lyric/31566643/Eminem/Psychopath+Killer'
]

# Функция для сбора текстов песен по ссылкам
def collect_lyrics(links):
    songs_lyrics = {}
    for link in links:
        soup = get_soup(link)
        song_name = soup.find('h1', {'class': 'lyric-title'}).text
        try:
            lyrics = soup.find('pre', {'id': 'lyric-body-text'}).text
            songs_lyrics[song_name] = lyrics
        except AttributeError:
            print(f'No lyrics for "{song_name}"')
        time.sleep(5)  # для предотвращения блокировки
    return songs_lyrics

# Функция для получения BeautifulSoup объекта из URL
def get_soup(path):
    r = requests.get(BASE_URL + path)
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup

# Собираем тексты песен для жанров "pop" и "rap"
pop_lyrics = collect_lyrics(pop_links)
rap_lyrics = collect_lyrics(rap_links)

# проверка  что количество песен в обоих жанрах равно
if len(pop_lyrics) == len(rap_lyrics):
    print(f'Оба жанра имеют {len(pop_lyrics)} песен.')
else:
    print('Количество песен в обоих жанрах неодинаково.')

# Объединяем тексты песен
all_lyrics = {**pop_lyrics, **rap_lyrics}

Оба жанра имеют 12 песен.


In [105]:
# Отобразил собранные программой-парсером песни
print(f'----- {len(all_lyrics)} songs: -----')
for song_name in all_lyrics:
    print(song_name)


----- 24 songs: -----
Guts Over Fear
Kamikaze
Fine Line
Not Alike
Normal
Stepping Stone
Greatest
Lucky You
The Ringer
Believe
Untouchable
River
Remind Me
Bad Husband
Tragic Endings
Amigo Bronco
Nowhere Fast
Heat
Offended
Need Me
In Your Head
Castle
Shady XV
Psychopath Killer


In [106]:
# Выводим первую песню собранную программой парсером
first_song_name, first_song_lyrics = list(all_lyrics.items())[0]
print(f'Первая песня:\n{first_song_name}\n\n{first_song_lyrics}')


Первая песня:
Guts Over Fear

Feels like a close, it’s coming to
F*ck am I gonna do?
It's too late to start over
This is the only thing I, thing I know

Sometimes I feel like all I ever do is
Find different ways to word the same, old song
Ever since I came along
From the day the song called ‘’Hi! My Name Is’’ dropped
Started thinking my name was fault
'Cause anytime things went wrong
I was the one who they would blame it on
The media made me the equivalent of a modern-day Genghis Khan
Tried to argue it was only entertainment, dawg
Gangsta? Naw, courageous balls
Had to change my style, they said I'm way too soft
And I sound like AZ and Nas, out came the claws
And the fangs been out since then
But up until the instant that I’ve been against it
It was ingrained in me that I wouldn’t amount to a shitstain I thought
No wonder I had to unlearn everything my brain was taught
Do I really belong in this game? I pondered
I just wanna play my part, should I make waves or not?
So back and forth in

In [107]:
#Разбил тексты песен на токены в качестве токенов использовал слова в NLTK
from nltk.tokenize import word_tokenize

# создаем словарь, в котором ключи - названия песен, а значения - списки токенов
song_tokens = {}

# разбиваем тексты песен на токены
for song_name, lyrics in all_lyrics.items():
    tokens = word_tokenize(lyrics, language='english')
    song_tokens[song_name] = tokens


In [108]:
# Выводим пример токенов для первой песни
print("Токены для первой песни:")
for token in song_tokens[list(all_lyrics.keys())[0]]:
    print(token)

Токены для первой песни:
Feels
like
a
close
,
it
’
s
coming
to
F
*
ck
am
I
gon
na
do
?
It
's
too
late
to
start
over
This
is
the
only
thing
I
,
thing
I
know
Sometimes
I
feel
like
all
I
ever
do
is
Find
different
ways
to
word
the
same
,
old
song
Ever
since
I
came
along
From
the
day
the
song
called
‘
’
Hi
!
My
Name
Is
’
’
dropped
Started
thinking
my
name
was
fault
'Cause
anytime
things
went
wrong
I
was
the
one
who
they
would
blame
it
on
The
media
made
me
the
equivalent
of
a
modern-day
Genghis
Khan
Tried
to
argue
it
was
only
entertainment
,
dawg
Gangsta
?
Naw
,
courageous
balls
Had
to
change
my
style
,
they
said
I
'm
way
too
soft
And
I
sound
like
AZ
and
Nas
,
out
came
the
claws
And
the
fangs
been
out
since
then
But
up
until
the
instant
that
I
’
ve
been
against
it
It
was
ingrained
in
me
that
I
wouldn
’
t
amount
to
a
shitstain
I
thought
No
wonder
I
had
to
unlearn
everything
my
brain
was
taught
Do
I
really
belong
in
this
game
?
I
pondered
I
just
wan
na
play
my
part
,
should
I
make
waves
or
not

In [109]:
#Исключил стоп слова знаков препинания из токенизированного текста
from nltk.corpus import stopwords
from string import punctuation

custom_stop_words = ["'m", "'re", "n't", "'s", 'na']  # Доп., стоп слова если необходимо
stop_words = set(stopwords.words('english') + custom_stop_words)

for song_name, tokens in song_tokens.items():
    tokens = [token.lower() for token in tokens]
    filtered_tokens = []
    for token in tokens:
        if token not in stop_words and token not in punctuation:
            filtered_tokens.append(token)
    song_tokens[song_name] = filtered_tokens


In [110]:
for song_name, tokens in song_tokens.items():
    print(f'{song_name}\n\n{tokens}\n')

Guts Over Fear

['feels', 'like', 'close', '’', 'coming', 'f', 'ck', 'gon', 'late', 'start', 'thing', 'thing', 'know', 'sometimes', 'feel', 'like', 'ever', 'find', 'different', 'ways', 'word', 'old', 'song', 'ever', 'since', 'came', 'along', 'day', 'song', 'called', '‘', '’', 'hi', 'name', '’', '’', 'dropped', 'started', 'thinking', 'name', 'fault', "'cause", 'anytime', 'things', 'went', 'wrong', 'one', 'would', 'blame', 'media', 'made', 'equivalent', 'modern-day', 'genghis', 'khan', 'tried', 'argue', 'entertainment', 'dawg', 'gangsta', 'naw', 'courageous', 'balls', 'change', 'style', 'said', 'way', 'soft', 'sound', 'like', 'az', 'nas', 'came', 'claws', 'fangs', 'since', 'instant', '’', 'ingrained', '’', 'amount', 'shitstain', 'thought', 'wonder', 'unlearn', 'everything', 'brain', 'taught', 'really', 'belong', 'game', 'pondered', 'wan', 'play', 'part', 'make', 'waves', 'back', 'forth', 'brain', 'tug', 'war', 'wages', '’', 'wan', 'seem', 'ungrateful', 'disrespect', 'artform', 'raised', 

In [111]:
#стемминг  это процесс получения корневой формы слова. Привел токены к корневой форме с помощью стемминга:
from nltk.stem.porter import PorterStemmer

# создаем стеммер
stemmer = PorterStemmer()

# производим стемминг для каждого токена в каждой песне
for song_name, tokens in song_tokens.items():
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    song_tokens[song_name] = stemmed_tokens

# выводим результат стемминга для первой песни
song_name, tokens = list(song_tokens.items())[0]
print(f'{song_name}\n\n{tokens}')


Guts Over Fear

['feel', 'like', 'close', '’', 'come', 'f', 'ck', 'gon', 'late', 'start', 'thing', 'thing', 'know', 'sometim', 'feel', 'like', 'ever', 'find', 'differ', 'way', 'word', 'old', 'song', 'ever', 'sinc', 'came', 'along', 'day', 'song', 'call', '‘', '’', 'hi', 'name', '’', '’', 'drop', 'start', 'think', 'name', 'fault', "'caus", 'anytim', 'thing', 'went', 'wrong', 'one', 'would', 'blame', 'media', 'made', 'equival', 'modern-day', 'genghi', 'khan', 'tri', 'argu', 'entertain', 'dawg', 'gangsta', 'naw', 'courag', 'ball', 'chang', 'style', 'said', 'way', 'soft', 'sound', 'like', 'az', 'na', 'came', 'claw', 'fang', 'sinc', 'instant', '’', 'ingrain', '’', 'amount', 'shitstain', 'thought', 'wonder', 'unlearn', 'everyth', 'brain', 'taught', 'realli', 'belong', 'game', 'ponder', 'wan', 'play', 'part', 'make', 'wave', 'back', 'forth', 'brain', 'tug', 'war', 'wage', '’', 'wan', 'seem', 'ungrat', 'disrespect', 'artform', 'rais', 'upon', 'sometim', 'got', 'ta', 'take', 'loss', 'peopl', 'r

In [112]:
from collections import Counter

# создаем словарь для хранения количества вхождений каждого слова в каждой песне
song_token_count = {}

# подсчитываем вхождения слов в токенах каждой песни
for song_name, tokens in song_tokens.items():
    token_count = Counter(tokens)
    song_token_count[song_name] = dict(token_count)

# выводим результат подсчета для первой песни
song_name, token_count = list(song_token_count.items())[0]
print(f'{song_name}\n\n{token_count}')


Guts Over Fear

{'feel': 6, 'like': 7, 'close': 3, '’': 25, 'come': 2, 'f': 3, 'ck': 3, 'gon': 4, 'late': 2, 'start': 5, 'thing': 6, 'know': 4, 'sometim': 2, 'ever': 3, 'find': 5, 'differ': 2, 'way': 7, 'word': 3, 'old': 2, 'song': 6, 'sinc': 2, 'came': 2, 'along': 1, 'day': 4, 'call': 1, '‘': 2, 'hi': 1, 'name': 2, 'drop': 1, 'think': 2, 'fault': 1, "'caus": 2, 'anytim': 1, 'went': 1, 'wrong': 1, 'one': 2, 'would': 1, 'blame': 1, 'media': 1, 'made': 8, 'equival': 1, 'modern-day': 1, 'genghi': 1, 'khan': 1, 'tri': 1, 'argu': 1, 'entertain': 1, 'dawg': 1, 'gangsta': 1, 'naw': 1, 'courag': 1, 'ball': 1, 'chang': 1, 'style': 1, 'said': 2, 'soft': 1, 'sound': 3, 'az': 1, 'na': 1, 'claw': 1, 'fang': 1, 'instant': 1, 'ingrain': 1, 'amount': 1, 'shitstain': 1, 'thought': 1, 'wonder': 1, 'unlearn': 1, 'everyth': 2, 'brain': 2, 'taught': 1, 'realli': 2, 'belong': 1, 'game': 1, 'ponder': 1, 'wan': 6, 'play': 2, 'part': 2, 'make': 7, 'wave': 1, 'back': 1, 'forth': 1, 'tug': 1, 'war': 1, 'wage': 1

In [113]:
# выводим результат подсчета для каждой песни
for song_name, token_count in song_token_count.items():
    print(f'{song_name}\n\n{token_count}\n')

Guts Over Fear

{'feel': 6, 'like': 7, 'close': 3, '’': 25, 'come': 2, 'f': 3, 'ck': 3, 'gon': 4, 'late': 2, 'start': 5, 'thing': 6, 'know': 4, 'sometim': 2, 'ever': 3, 'find': 5, 'differ': 2, 'way': 7, 'word': 3, 'old': 2, 'song': 6, 'sinc': 2, 'came': 2, 'along': 1, 'day': 4, 'call': 1, '‘': 2, 'hi': 1, 'name': 2, 'drop': 1, 'think': 2, 'fault': 1, "'caus": 2, 'anytim': 1, 'went': 1, 'wrong': 1, 'one': 2, 'would': 1, 'blame': 1, 'media': 1, 'made': 8, 'equival': 1, 'modern-day': 1, 'genghi': 1, 'khan': 1, 'tri': 1, 'argu': 1, 'entertain': 1, 'dawg': 1, 'gangsta': 1, 'naw': 1, 'courag': 1, 'ball': 1, 'chang': 1, 'style': 1, 'said': 2, 'soft': 1, 'sound': 3, 'az': 1, 'na': 1, 'claw': 1, 'fang': 1, 'instant': 1, 'ingrain': 1, 'amount': 1, 'shitstain': 1, 'thought': 1, 'wonder': 1, 'unlearn': 1, 'everyth': 2, 'brain': 2, 'taught': 1, 'realli': 2, 'belong': 1, 'game': 1, 'ponder': 1, 'wan': 6, 'play': 2, 'part': 2, 'make': 7, 'wave': 1, 'back': 1, 'forth': 1, 'tug': 1, 'war': 1, 'wage': 1

In [114]:
# создаем пустой словарь где ключи будут названиями песен а значения  словаря BoW для каждой песни
song_bow = {}

# проходимся по всем песням и создадим BoW для каждой
for song_name, tokens in song_tokens.items():
    # используем Counter для подсчета частоты каждого слова в песне
    word_count = Counter(tokens)
    song_bow[song_name] = dict(word_count)

# Преобразуем полученный словарь в DataFrame для удобства работы
bow_df = pd.DataFrame(song_bow).T.fillna(0)

# у нас есть таблица BoW для каждой песни где строки  названия песен а столбцы  слова и их частоты в песне

bow_df

,feel,like,close,’,come,f,ck,gon,late,start,...,dahmer-lik,ridin,lock,bonni,clyde,uncl,ronni,burst,asylum,dine
Guts Over Fear,6.0,7.0,3.0,25.0,2.0,3.0,3.0,4.0,2.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kamikaze,1.0,8.0,1.0,0.0,2.0,0.0,0.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fine Line,3.0,8.0,1.0,0.0,4.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Not Alike,2.0,14.0,1.0,0.0,3.0,6.0,6.0,3.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Normal,0.0,9.0,0.0,0.0,3.0,2.0,2.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Stepping Stone,5.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Greatest,4.0,19.0,0.0,0.0,1.0,4.0,4.0,4.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lucky You,1.0,10.0,0.0,0.0,1.0,9.0,9.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The Ringer,2.0,16.0,1.0,0.0,1.0,7.0,7.0,9.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Believe,1.0,8.0,2.0,0.0,0.0,2.0,2.0,2.0,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [115]:
# Заменяем все ненулевые значения в таблице BoW на 1 (если слово встречается то 1 иначе 0)
bow_df = bow_df.map(lambda x: 1 if x > 0 else 0)

# Выводим  обновленную таблицу BoW где теперь 1 указывает на наличие слова в песне а 0  на отсутствие
bow_df


,feel,like,close,’,come,f,ck,gon,late,start,...,dahmer-lik,ridin,lock,bonni,clyde,uncl,ronni,burst,asylum,dine
Guts Over Fear,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Kamikaze,1,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Fine Line,1,1,1,0,1,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
Not Alike,1,1,1,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Normal,0,1,0,0,1,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Stepping Stone,1,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Greatest,1,1,0,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
Lucky You,1,1,0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
The Ringer,1,1,1,0,1,1,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
Believe,1,1,1,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [116]:
# Вычисляем TF для каждой песни
total_word_count = bow_df.sum(axis=1)  # Общее количество слов в каждой песне
tf_df = bow_df.divide(total_word_count, axis=0)

# Выводим таблицу TF
tf_df


,feel,like,close,’,come,f,ck,gon,late,start,...,dahmer-lik,ridin,lock,bonni,clyde,uncl,ronni,burst,asylum,dine
Guts Over Fear,0.003953,0.003953,0.003953,0.003953,0.003953,0.003953,0.003953,0.003953,0.003953,0.003953,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Kamikaze,0.004149,0.004149,0.004149,0.000000,0.004149,0.000000,0.000000,0.004149,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Fine Line,0.004082,0.004082,0.004082,0.000000,0.004082,0.004082,0.004082,0.000000,0.004082,0.004082,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Not Alike,0.002488,0.002488,0.002488,0.000000,0.002488,0.002488,0.002488,0.002488,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Normal,0.000000,0.004049,0.000000,0.000000,0.004049,0.004049,0.004049,0.000000,0.000000,0.004049,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Stepping Stone,0.003891,0.003891,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.003891,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Greatest,0.003135,0.003135,0.000000,0.000000,0.003135,0.003135,0.003135,0.003135,0.003135,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Lucky You,0.002667,0.002667,0.000000,0.000000,0.002667,0.002667,0.002667,0.002667,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
The Ringer,0.002079,0.002079,0.002079,0.000000,0.002079,0.002079,0.002079,0.002079,0.000000,0.002079,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Believe,0.003155,0.003155,0.003155,0.000000,0.000000,0.003155,0.003155,0.003155,0.003155,0.003155,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [117]:
# Вычисляем IDF для каждого слова
total_song_count = len(bow_df)  # Общее количество песен
word_presence = bow_df.astype(bool).sum(axis=0)  # Количество песен, в которых встречается каждое слово
idf_series = word_presence.apply(lambda x: math.log10(total_song_count / x))

# Вывод слов и их соответствующие IDF значения
idf_series


feel      0.101458
like      0.018483
close     0.338819
’         0.778151
come      0.079181
            ...   
uncl      1.380211
ronni     1.380211
burst     1.380211
asylum    1.380211
dine      1.380211
Length: 3212, dtype: float64

In [118]:
# Перемножаем значения TF и IDF
tfidf_matrix = tf_df * idf_series

# Выводим матрицу TF-IDF
tfidf_matrix


,feel,like,close,’,come,f,ck,gon,late,start,...,dahmer-lik,ridin,lock,bonni,clyde,uncl,ronni,burst,asylum,dine
Guts Over Fear,0.000401,0.000073,0.001339,0.003076,0.000313,0.000401,0.000401,0.000925,0.002115,0.001339,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Kamikaze,0.000421,0.000077,0.001406,0.000000,0.000329,0.000000,0.000000,0.000971,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Fine Line,0.000414,0.000075,0.001383,0.000000,0.000323,0.000414,0.000414,0.000000,0.002184,0.001383,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Not Alike,0.000252,0.000046,0.000843,0.000000,0.000197,0.000252,0.000252,0.000582,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Normal,0.000000,0.000075,0.000000,0.000000,0.000321,0.000411,0.000411,0.000000,0.000000,0.001372,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Stepping Stone,0.000395,0.000072,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001318,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Greatest,0.000318,0.000058,0.000000,0.000000,0.000248,0.000318,0.000318,0.000734,0.001677,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Lucky You,0.000271,0.000049,0.000000,0.000000,0.000211,0.000271,0.000271,0.000624,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
The Ringer,0.000211,0.000038,0.000704,0.000000,0.000165,0.000211,0.000211,0.000487,0.000000,0.000704,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Believe,0.000320,0.000058,0.001069,0.000000,0.000000,0.000320,0.000320,0.000738,0.001688,0.001069,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [119]:
# Выберем первые 12 поп-песен
pop_songs_tfidf = tfidf_matrix.iloc[:12]

# Выведем выбранные поп-песни
pop_songs_tfidf


,feel,like,close,’,come,f,ck,gon,late,start,...,dahmer-lik,ridin,lock,bonni,clyde,uncl,ronni,burst,asylum,dine
Guts Over Fear,0.000401,0.000073,0.001339,0.003076,0.000313,0.000401,0.000401,0.000925,0.002115,0.001339,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Kamikaze,0.000421,0.000077,0.001406,0.000000,0.000329,0.000000,0.000000,0.000971,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fine Line,0.000414,0.000075,0.001383,0.000000,0.000323,0.000414,0.000414,0.000000,0.002184,0.001383,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Not Alike,0.000252,0.000046,0.000843,0.000000,0.000197,0.000252,0.000252,0.000582,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Normal,0.000000,0.000075,0.000000,0.000000,0.000321,0.000411,0.000411,0.000000,0.000000,0.001372,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Stepping Stone,0.000395,0.000072,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001318,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Greatest,0.000318,0.000058,0.000000,0.000000,0.000248,0.000318,0.000318,0.000734,0.001677,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Lucky You,0.000271,0.000049,0.000000,0.000000,0.000211,0.000271,0.000271,0.000624,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
The Ringer,0.000211,0.000038,0.000704,0.000000,0.000165,0.000211,0.000211,0.000487,0.000000,0.000704,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Believe,0.000320,0.000058,0.001069,0.000000,0.000000,0.000320,0.000320,0.000738,0.001688,0.001069,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [120]:
# Выберем первые 12 реп-песен
rap_songs_tfidf = tfidf_matrix.iloc[12:24]
rap_songs_tfidf

,feel,like,close,’,come,f,ck,gon,late,start,...,dahmer-lik,ridin,lock,bonni,clyde,uncl,ronni,burst,asylum,dine
Remind Me,0.000000,0.000081,0.000000,0.000000,0.000346,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Bad Husband,0.000000,0.000082,0.001506,0.000000,0.000352,0.000451,0.000451,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Tragic Endings,0.000551,0.000100,0.000000,0.000000,0.000430,0.000551,0.000551,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Amigo Bronco,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Nowhere Fast,0.000495,0.000090,0.000000,0.000000,0.000386,0.000495,0.000495,0.001142,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Heat,0.000373,0.000068,0.000000,0.000000,0.000291,0.000373,0.000373,0.000861,0.001967,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Offended,0.000222,0.000041,0.000743,0.000000,0.000174,0.000222,0.000222,0.000513,0.000000,0.000743,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Need Me,0.000676,0.000123,0.000000,0.000000,0.000528,0.000000,0.000000,0.000000,0.000000,0.002259,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
In Your Head,0.000686,0.000125,0.000000,0.000000,0.000000,0.000686,0.000686,0.000000,0.003616,0.002289,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Castle,0.000449,0.000082,0.001499,0.003443,0.000350,0.000449,0.000449,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [121]:
# Объединим выбранные песни в один список
songs_for_classifier = pd.concat([pop_songs_tfidf, rap_songs_tfidf])

# Выведем выбранные песни для классификатора
songs_for_classifier

,feel,like,close,’,come,f,ck,gon,late,start,...,dahmer-lik,ridin,lock,bonni,clyde,uncl,ronni,burst,asylum,dine
Guts Over Fear,0.000401,0.000073,0.001339,0.003076,0.000313,0.000401,0.000401,0.000925,0.002115,0.001339,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Kamikaze,0.000421,0.000077,0.001406,0.000000,0.000329,0.000000,0.000000,0.000971,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Fine Line,0.000414,0.000075,0.001383,0.000000,0.000323,0.000414,0.000414,0.000000,0.002184,0.001383,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Not Alike,0.000252,0.000046,0.000843,0.000000,0.000197,0.000252,0.000252,0.000582,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Normal,0.000000,0.000075,0.000000,0.000000,0.000321,0.000411,0.000411,0.000000,0.000000,0.001372,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Stepping Stone,0.000395,0.000072,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.001318,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Greatest,0.000318,0.000058,0.000000,0.000000,0.000248,0.000318,0.000318,0.000734,0.001677,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Lucky You,0.000271,0.000049,0.000000,0.000000,0.000211,0.000271,0.000271,0.000624,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
The Ringer,0.000211,0.000038,0.000704,0.000000,0.000165,0.000211,0.000211,0.000487,0.000000,0.000704,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Believe,0.000320,0.000058,0.001069,0.000000,0.000000,0.000320,0.000320,0.000738,0.001688,0.001069,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [122]:
# Создаем матрицу признаков X
X = songs_for_classifier

# Создаем вектор меток y, где 0 обозначает поп-песни, а 1 обозначает реп-песни
y = [0] * 12 + [1] * 12

# Выведем матрицу признаков X и вектор меток y
print("Матрица признаков (X):")
print(X)
print("\nВектор меток (y):")
print(y)

Матрица признаков (X):
                       feel      like     close         ’      come         f  \
Guts Over Fear     0.000401  0.000073  0.001339  0.003076  0.000313  0.000401   
Kamikaze           0.000421  0.000077  0.001406  0.000000  0.000329  0.000000   
Fine Line          0.000414  0.000075  0.001383  0.000000  0.000323  0.000414   
Not Alike          0.000252  0.000046  0.000843  0.000000  0.000197  0.000252   
Normal             0.000000  0.000075  0.000000  0.000000  0.000321  0.000411   
Stepping Stone     0.000395  0.000072  0.000000  0.000000  0.000000  0.000000   
Greatest           0.000318  0.000058  0.000000  0.000000  0.000248  0.000318   
Lucky You          0.000271  0.000049  0.000000  0.000000  0.000211  0.000271   
The Ringer         0.000211  0.000038  0.000704  0.000000  0.000165  0.000211   
Believe            0.000320  0.000058  0.001069  0.000000  0.000000  0.000320   
Untouchable        0.000258  0.000047  0.000862  0.001980  0.000201  0.000258   
River

In [159]:
from sklearn.model_selection import train_test_split

# Разделяем данные на тренировочный (70%) и тестовый (30%) наборы для всех классификаторов
X_train_svc, X_test_svc, y_train_svc, y_test_svc = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_knn, X_test_knn, y_train_knn, y_test_knn = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_rf, X_test_rf, y_train_rf, y_test_rf = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_gb, X_test_gb, y_train_gb, y_test_gb = train_test_split(X, y, test_size=0.3, random_state=42)

# Выводим размеры наборов данных для всех классификаторов
print("Размер тренировочных данных (X_train_svc, y_train_svc):", X_train_svc.shape, len(y_train_svc))
print("Размер тестовых данных (X_test_svc, y_test_svc):", X_test_svc.shape, len(y_test_svc))
print("Размер тренировочных данных (X_train_knn, y_train_knn):", X_train_knn.shape, len(y_train_knn))
print("Размер тестовых данных (X_test_knn, y_test_knn):", X_test_knn.shape, len(y_test_knn))
print("Размер тренировочных данных (X_train_rf, y_train_rf):", X_train_rf.shape, len(y_train_rf))
print("Размер тестовых данных (X_test_rf, y_test_rf):", X_test_rf.shape, len(y_test_rf))
print("Размер тренировочных данных (X_train_gb, y_train_gb):", X_train_gb.shape, len(y_train_gb))
print("Размер тестовых данных (X_test_gb, y_test_gb):", X_test_gb.shape, len(y_test_gb))


Размер тренировочных данных (X_train_svc, y_train_svc): (16, 3212) 16
Размер тестовых данных (X_test_svc, y_test_svc): (8, 3212) 8
Размер тренировочных данных (X_train_knn, y_train_knn): (16, 3212) 16
Размер тестовых данных (X_test_knn, y_test_knn): (8, 3212) 8
Размер тренировочных данных (X_train_rf, y_train_rf): (16, 3212) 16
Размер тестовых данных (X_test_rf, y_test_rf): (8, 3212) 8
Размер тренировочных данных (X_train_gb, y_train_gb): (16, 3212) 16
Размер тестовых данных (X_test_gb, y_test_gb): (8, 3212) 8


In [160]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Создаем экземпляр классификатора метода опорных векторов
svm_classifier = SVC()

# Создаем экземпляр классификатора метода k-ближайших соседей (KNN)
knn_classifier = KNeighborsClassifier()

# Создаем экземпляр классификатора Random Forest
rf_classifier = RandomForestClassifier()

# Создаем экземпляр классификатора Gradient Boosting
gb_classifier = GradientBoostingClassifier()


# Выводим информацию о созданных классификаторах
print("Создан классификатор метода опорных векторов (SVM):", svm_classifier)
print("Создан классификатор метода k-ближайших соседей (KNN):", knn_classifier)
print("Создан классификатор метода Random Forest (RF):", rf_classifier)
print("Создан классификатор метода Gradient Boosting  (GB):", gb_classifier)

Создан классификатор метода опорных векторов (SVM): SVC()
Создан классификатор метода k-ближайших соседей (KNN): KNeighborsClassifier()
Создан классификатор метода Random Forest (RF): RandomForestClassifier()
Создан классификатор метода Gradient Boosting  (GB): GradientBoostingClassifier()


In [168]:
# Отключил предупреждения из за устаревших library и функций 
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from sklearn.metrics import accuracy_score

# Обучаем SVM
svm_classifier.fit(X_train, y_train)

# Обучаем KNN
knn_classifier.fit(X_train, y_train)

# Обучаем Random Forest
rf_classifier.fit(X_train, y_train)

# Обучаем Gradient Boosting
gb_classifier.fit(X_train, y_train)

# Предсказываем метки для тестовых данных
svm_predictions = svm_classifier.predict(X_test)
knn_predictions = knn_classifier.predict(X_test)
rf_predictions = rf_classifier.predict(X_test)
gb_predictions = gb_classifier.predict(X_test)

# Оцениваем точность классификаторов
svm_accuracy = accuracy_score(y_test, svm_predictions)
knn_accuracy = accuracy_score(y_test, knn_predictions)
rf_accuracy = accuracy_score(y_test, rf_predictions)
gb_accuracy = accuracy_score(y_test, gb_predictions)

print("Точность SVM:", svm_accuracy)
print("Точность KNN:", knn_accuracy)
print("Точность Random Forest:", rf_accuracy)
print("Точность Gradient Boosting:", gb_accuracy)

Точность SVM: 0.375
Точность KNN: 0.625
Точность Random Forest: 0.75
Точность Gradient Boosting: 0.375


In [169]:
from sklearn.exceptions import UndefinedMetricWarning
import warnings 
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

# Функция для подсчета метрик: точности, полноты, специфичности, прецизионности
def calculate_metrics(y_true, y_pred):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=UndefinedMetricWarning)
        # Вычисляем точность
        accuracy = accuracy_score(y_true, y_pred)

        # Вычисляем прецизионность (precision) с установкой zero_division=1
        precision = precision_score(y_true, y_pred, zero_division=1)

        # Вычисляем полноту (recall)
        recall = recall_score(y_true, y_pred)

        # Вычисляем матрицу ошибок (confusion matrix) и извлекаем true negatives, false positives, false negatives и true positives
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

        # Вычисляем специфичность (specificity)
        specificity = tn / (tn + fp)

    return accuracy, precision, recall, specificity

# Пример использования функции:
y_true = [1, 0, 1, 1, 0, 0, 1, 0]  # Реальные метки класса (истинные значения)
y_pred = [1, 0, 1, 0, 1, 0, 1, 1]  # Предсказанные метки класса

# Вызываем функцию для подсчета метрик
accuracy, precision, recall, specificity = calculate_metrics(y_true, y_pred)

# Выводим результаты
print(f'Accuracy: {accuracy}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'Specificity: {specificity}')


Accuracy: 0.625
Precision: 0.6
Recall: 0.75
Specificity: 0.5


In [170]:
# использование функции для SVM
accuracy_svm, precision_svm, recall_svm, specificity_svm = calculate_metrics(y_test, svm_predictions)
print("Метрики для SVM:")
print(f'Accuracy: {accuracy_svm}')
print(f'Precision: {precision_svm}')
print(f'Recall: {recall_svm}')
print(f'Specificity: {specificity_svm}\n')

# использование функции для KNN
accuracy_knn, precision_knn, recall_knn, specificity_knn = calculate_metrics(y_test, knn_predictions)
print("Метрики для KNN:")
print(f'Accuracy: {accuracy_knn}')
print(f'Precision: {precision_knn}')
print(f'Recall: {recall_knn}')
print(f'Specificity: {specificity_knn}\n')

# использование функции для Random Forest
accuracy_rf, precision_rf, recall_rf, specificity_rf = calculate_metrics(y_test, rf_predictions)
print("Метрики для Random Forest:")
print(f'Accuracy: {accuracy_rf}')
print(f'Precision: {precision_rf}')
print(f'Recall: {recall_rf}')
print(f'Specificity: {specificity_rf}\n')

# использование функции для Gradient Boosting
accuracy_gb, precision_gb, recall_gb, specificity_gb = calculate_metrics(y_test, gb_predictions)
print("Метрики для Gradient Boosting:")
print(f'Accuracy: {accuracy_gb}')
print(f'Precision: {precision_gb}')
print(f'Recall: {recall_gb}')
print(f'Specificity: {specificity_gb}')


Метрики для SVM:
Accuracy: 0.375
Precision: 0.375
Recall: 1.0
Specificity: 0.0

Метрики для KNN:
Accuracy: 0.625
Precision: 1.0
Recall: 0.0
Specificity: 1.0

Метрики для Random Forest:
Accuracy: 0.75
Precision: 0.6
Recall: 1.0
Specificity: 0.6

Метрики для Gradient Boosting:
Accuracy: 0.375
Precision: 0.25
Recall: 0.3333333333333333
Specificity: 0.4



## В ходе выполнения данной работы было выполнено:




  
  
  1. Собраны тексты песен из веб-сайта lyrics.com для двух жанров.
   
   2.  Проведена фильтрация, токенизация и нормализация текстовых данных путем удаления стоп-слов и пунктуации.

   3. Применен метод стемминга для приведения слов к корневой форме.

   4. Создан словарь, в котором ключи представляют названия песен, а значения - списки токенов.

   5. Подсчитаны частоты вхождений каждого слова в каждой песне и сохранены в словаре.

   6. Созданы таблицы Bag of Words (BoW) для каждой песни.

   7. Преобразованы значения BoW в 1 (если слово встречается) и 0 (если отсутствует) для каждой песни.

   8. Вычислена метрика Term Frequency-Inverse Document Frequency (TF-IDF) для каждой песни.

   9. Выбраны песни из двух жанров для использования в классификации.

   10. Созданы матрицы признаков (X) и векторы меток (y) для классификации.

   11. Разделены данные на тренировочный и тестовый наборы (70% и 30% соответственно) для четырех различных классификаторов: SVM, KNN, Random Forest и Gradient Boosting.

   12. Созданы экземпляры классификаторов для каждого метода.

   13. Обучены классификаторы на тренировочных данных.

   14. Предсказаны метки для тестовых данных и оценена точность (accuracy) классификации.

   15. Оценены дополнительные метрики, включая precision, recall, и specificity для каждого классификатора.

   16. Сравнены результаты классификации между разными классификаторами.

   17. Использованы четыре бинарных классификатора для задачи классификации текстовых данных.

   18. Метрика TF-IDF была реализована с использованием библиотеки scikit-learn.

   19. Все метрики были подсчитаны для оценки производительности классификаторов.

   20. Работа включает в себя обработку текстовых данных, создание признаков и оценку эффективности классификации.

## Заключение

    Главным выводом данной работы является то, что проведенный анализ текстов песен и их классификация на жанры при помощи различных методов классификации может быть полезным инструментом в музыкальной индустрии. Несмотря на низкие показатели точности в некоторых методах, этот подход имеет потенциал для применения в создании персонализированных музыкальных рекомендаций, музыкального маркетинга и исследований. Работа также подчеркивает важность дальнейшего совершенствования методов и расширения датасетов для улучшения точности и применимости таких анализов в будущем.